# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: <strong>REMOVE</strong> any print statements added to cells with "#export" that are used for debugging purposes befrore submitting because they will crash the autograder in Gradescope. Any additional cells can be used for testing purposes at the bottom. 
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove any comment that says "#export" because that will crash the autograder in Gradescope. We use this comment to export your code in these cells for grading.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> import any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance. If you don't get any output, make sure your Notebook's Kernel is set to "PySpark" in the top right corner.
</div>

In [1]:
sc

Attach to a cluster to execute a cell.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [1]:
#export
from pyspark.sql.functions import col
from pyspark.sql import *

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1666624649294_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://cse6242-hw3-q3"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp, lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function. Do <strong>NOT</strong> remove the #export comment from each of the code blocks either. This can prevent your code from being converted to a python file.
</div>

## 3a. [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [3]:
#export
def user():
    # Returns a string consisting of your GT username.
    return 'ywang4068'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3b. [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips greater than or equal to 2 miles.

In [4]:
#export
def long_trips(trips):
    # Returns a Dataframe (trips) with Schema the same as :trips:
    return trips.filter(col('trip_distance') >= 2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3c. [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [27]:
#export
def manhattan_trips(trips, lookup):
    # Returns a Dataframe (mtrips) with Schema: DOLocationID, pcount
    df = lookup.join(trips, lookup.LocationID == trips.DOLocationID)\
                .filter(col('Borough') == 'Manhattan')
    return df.groupBy('DOLocationID').agg({'passenger_count': 'sum'})\
              .withColumnRenamed('sum(passenger_count)', 'pcount')\
              .orderBy(col('pcount').desc()).limit(20)

             

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3d. [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [28]:
#export
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe (wp) with Schema: PULocationID, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    df_trips = trips.groupBy('PULocationID').agg({'total_amount': 'avg', 'PULocationID': 'count'})\
                    .withColumnRenamed('avg(total_amount)', 'avg')\
                    .withColumnRenamed('count(PULocationID)', 'tcount')\
                    .withColumn('avg', col('avg').cast('decimal(18,10)'))
    df_mtrips = trips.join(mtrips, 'DOLocationID', 'leftsemi').groupBy('PULocationID').count()
    df = df_trips.join(df_mtrips, 'PULocationID')\
                .withColumn('proportion', (col('count')/col('tcount')).cast('decimal(18,10)'))\
                .withColumn('weighted_profit', (col('proportion') * col('avg')).cast('decimal(18,10)'))
    return df.select('PULocationID', 'weighted_profit')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3e. [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [29]:
#export
def final_output(wp, lookup): 
    # Returns a Dataframe (final) with Schema: Zone, Borough, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    df = wp.join(lookup, wp.PULocationID == lookup.LocationID)\
            .orderBy(col('weighted_profit').desc()).limit(20)
    return df.select('Zone','Borough', 'weighted_profit')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading.
</div>

In [31]:
load_data(size='large')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Trip Count:  187203269
(DataFrame[VendorID: string, tpep_pickup_datetime: string, tpep_dropoff_datetime: string, passenger_count: string, trip_distance: string, RatecodeID: string, store_and_fwd_flag: string, PULocationID: string, DOLocationID: string, payment_type: string, fare_amount: string, extra: string, mta_tax: string, tip_amount: string, tolls_amount: string, improvement_surcharge: string, total_amount: string], DataFrame[LocationID: int, Borough: string, Zone: string, service_zone: string])

In [32]:
bucket = 's3://cse6242-ywang4068/q3_output_large_1'
# main('small',bucket)
main('large', bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ywang4068
Trip Count:  187203269
+--------------------+-------------+---------------+
|                Zone|      Borough|weighted_profit|
+--------------------+-------------+---------------+
|        Baisley Park|       Queens|  29.3604557817|
|Flushing Meadows-...|       Queens|  27.3048457358|
|       South Jamaica|       Queens|  26.2949162398|
|     Randalls Island|    Manhattan|  24.1509899399|
|        Astoria Park|       Queens|  21.7064171100|
|Briarwood/Jamaica...|       Queens|  19.9450646302|
|Springfield Garde...|       Queens|  19.4683092894|
|             Jamaica|       Queens|  19.2839430022|
|              Corona|       Queens|  18.2287692504|
|   LaGuardia Airport|       Queens|  18.1813388077|
|         Jamaica Bay|       Queens|  17.1005294451|
|             Maspeth|       Queens|  17.0054506402|
|Eltingville/Annad...|Staten Island|  16.8377647549|
|         JFK Airport|       Queens|  16.7777253477|
|        Battery Park|    Manhattan|  12.8497803121|
| Morningside